## **Covid-19 vaccination progress**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

This notebook aims to visualize the vaccination progress around the world. I start by doing my own calculation of the variable **daily_vaccinations**. The first visualization compares two variables in the dataset that can track the progress in different ways: **daily_vaccinations** and **daily_vaccinations_raw**. The second visualization compares groups of countries. The third one shows which vaccine is being used in which country. These visualizations were created with `plotly`.

In [ ]:
df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

There are several options to track the vaccination progress. One could pick almost every variable in the dataset and plot a line chart to show how many vaccinations or vaccinated people are there in each country. However, checking for missing values points to **daily_vaccinations** as the most appropriate variable for this analysis. According to *Our World in Data*, this variable measures "*new doses administered per day (7-day smoothed)*". However, there is also **daily_vaccinations_raw**, which measures the "*daily change in the total number of doses administered*". In principle, it would be better to use the latter, since it is the exact number of doses reported by each country, whereas the former is an average. However, due to the number of missing values in **daily_vaccinations_raw**, it is clear that **daily_vaccinations** offers a better picture of the vaccination progress.

Although **daily_vaccinations** is in the dataset, I wanted to see how this variable was calculated. I saw an explanatory [post](https://github.com/owid/covid-19-data/issues/333#issuecomment-763015298) on Github that gave me an idea on how to calculate these values, but I wanted to see if could come up with the same result on Python.

### **Comparing daily_vaccinations and daily_vaccinations_raw**

In this part, the approach is to use the values from **total_vaccinations** to calculate **daily_vaccinations**. In general terms, I follow the next steps:
* Calculate the difference between consecutive non-missing values in **total_vaccinations**
* Divide this difference by the number of consecutive missing values to get an estimated proportional value.
* Find the moving average (**MA**) of seven days of the estimated proportional value.


In [ ]:
# (1) Create a copy from the original df and fill gaps in total_vaccinations
dvac = df[['country', 'date', 'total_vaccinations', 
           'daily_vaccinations_raw', 'daily_vaccinations']].copy() #using copy to avoid SettingwithCopyWarning 

# Finding same consecutive values in 'total_vaccinations'
dvac['csc_value'] = dvac.total_vaccinations.groupby([dvac['country'],dvac['total_vaccinations']]).diff(1) == 0

# Filling gaps of missing values between same consecutive values in 'csc_value' and 'total_vaccinations'
dvac['csc_value'].fillna(method='bfill', inplace=True)

dvac.loc[dvac['csc_value'] == True, 'total_vaccinations'] = ((dvac['total_vaccinations'].groupby([dvac['country'], dvac['csc_value']]))
                                                        .transform(lambda x:x.bfill()))

# (2) Creates new variable from 'total_vaccinations' filling gaps of missing values
dvac['filltv'] = (dvac['total_vaccinations'].groupby(dvac['country'])               
               .transform(lambda x:x.bfill())) 

# (3) Counting missing values in 'total vaccinations'
dvac['nan_values'] = (dvac.total_vaccinations.isnull().astype(int).groupby(dvac.total_vaccinations.notnull().astype(int).cumsum())
                      .transform('sum')
                      .transform(lambda x:x+1 if x != 0 else 0) #adds one to take into account the non-missing value
                      .shift(1))

# (4) Calculates difference from consecutive non-missing values in 'filltv'
dvac.loc[dvac.nan_values != 0, 'diff_filltv'] = ((dvac['filltv'].groupby([dvac['country']]).diff(-1)*(-1)) #multiply by minus 1 to get positive result.
                                             .replace(-0, np.nan) 
                                             .shift(1) 
                                             .groupby([dvac['country']]).transform(lambda x:x.ffill()))

# (5) Divides the difference by the number of missing values
dvac['avg_filltv'] = (dvac['diff_filltv']/dvac['nan_values'])

# (6) Substitutes values with data from 'daily_vaccinations_raw', whenever it finds non-missing values
dvac.loc[dvac.nan_values == 0, 'avg_filltv'] = dvac['daily_vaccinations_raw']

# Sets values to zero based on variable 'csc_value'
dvac.loc[dvac.csc_value == True, 'avg_filltv'] = 0

# (7) moving avg
dvac['MA'] = round(dvac.groupby('country')['avg_filltv'] 
                   .rolling(window=7, min_periods=1) #takes 7 values. minimum 1 value for first values
                   .mean(), 0).reset_index(0,drop=True)

# If 'filltv' is a missing value, 'MA' also turns to a missing value
dvac.loc[dvac.filltv.isna(), 'MA'] = np.nan

At the end, the variable **MA** -the moving average- and **total_vaccinations** are basically the same. There is a difference in some values due to rounding errors, but it is not a significant change from the original variable.

In [ ]:
dvac[['country','date','daily_vaccinations','MA']].head(10)

In [ ]:
dvac['diff'] = dvac['MA'] - dvac['daily_vaccinations']
dvac.loc[dvac['diff'].notna() & dvac['diff'] != 0, ['country','date', 'daily_vaccinations', 'MA', 'diff']]

### **Visualization: daily_vaccinations and daily_vaccinations_raw**

The goal of this visualization is to see how using **daily_vaccinations** smooths the representation of this dataset. In some cases, where there is also full data for **daily_vaccinations_raw**, is interesting to see the difference between the two variables. 

In [ ]:
def country_heatmap(country, df=dvac):
    df = df.loc[df['country'] == country].copy()
    
    # time period
    df['date'] = pd.to_datetime(df['date']).apply(lambda x: x.date())
    start_date = min(df['date'])
    last_date = max(df['date'])
    timeperiod = last_date-start_date
    
    # weekdays and week number
    country_calendar = [start_date + datetime.timedelta(i) for i in range(timeperiod.days+1)] #list with days in timeperiod
    weekdays = [i.weekday() for i in country_calendar]
    weeknumber = [(i.strftime('%V')) for i in country_calendar]
    
    # daily vaccinations data
    vc = df['daily_vaccinations']
    vcr = df['daily_vaccinations_raw']    
    text = [str(i) for i in country_calendar]
    
    # subplots
    fig = make_subplots(1,2, 
                        shared_yaxes=False,
                        subplot_titles=('daily_vaccinations','daily_vaccinations_raw'))
    
    #daily_vaccinations heatmap
    fig.add_trace(
        go.Heatmap(
            x=weekdays,
            y=weeknumber,
            z=vc,
            text=text,
            xgap=2,
            ygap=2,
            showscale=False,
            colorscale='viridis',
            hovertemplate='Weekday: %{x}<br>Week number: %{y}<br>Vaccinations: %{z}<br>Date: %{text}<extra></extra>'),1,1) 
    
    #daily_vaccinations_raw heatmap
    fig.add_trace(
        go.Heatmap(
            x=weekdays,
            y=weeknumber,
            z=vcr,
            text=text,
            xgap=2,
            ygap=2,
            showscale=False,
            colorscale='viridis',
            hovertemplate='Weekday: %{x}<br>Week number: %{y}<br>Vaccinations: %{z}<br>Date: %{text}<extra></extra>'),1,2)
    
    fig.update_layout(
        title = country,
        title_font_size = 20,
        title_y = 0.98,        
        plot_bgcolor = ('rgb(255,255,255)'),
        width=800, 
        height=600 
        )
    fig.layout.annotations[0].update(y=1.05)
    fig.layout.annotations[1].update(y=1.05)
    fig.update_xaxes(
        tickmode="array",
        ticktext=["Mon","Tue","Wed","Thu","Fri","Sat","Sun"],
        tickvals=[0,1,2,3,4,5,6],
        title="",
        side="top"
        )
    fig.update_yaxes(
        title="Week Nr.",
        autorange="reversed"
        )    
    fig.show()


The first example is the United States. The vaccination began in the last two weeks of 2020 and reached its peak during the 15th week of 2021, which was around mid-April. This is clearly illustrated in the heat map to the left. However, in the heat map to the right, it is possible to distinguish another trend. Most of the vaccinations appear to be happening towards the weekend, which is more evident throughout the months of March, April and May.

In [ ]:
country_heatmap('United States')

For Germany, it appears June was the period with the highest daily vaccination average. In contrast to the United States, most of the vaccinations happened in the middle of the week, whereas the weekends are the days with the less number of vaccinations.

In [ ]:
country_heatmap('Germany')

Unfortunately, these comparisons do not work for every country, because of the missing values. In the case of Brazil, it's clear that the best option to visualize the vaccination progress in heat maps is by choosing the variable **daily_vaccinations**. It's difficult to identify trends in the heat map to the right. There are a lot of countries full of missing values in the variable **daily_vaccinations_raw**. 

In [ ]:
country_heatmap('Brazil')

### **Progress by region - North America and South America**

This a line chart comparing a group of countries using the variable **people_fully_vaccinated_per_hundred**. I created a group for North America and another for South America.

In [ ]:
### Find the max number of people vaccinated per 100 as of the most recent date
# Columns for country, date and total vaccinates
total_per_country = df[['country', 'date', 'people_fully_vaccinated_per_hundred']].copy()

# Filling empty values by group with foward fill (ffill)
total_per_country['people_fully_vaccinated_per_hundred'] = (total_per_country['people_fully_vaccinated_per_hundred'].groupby(total_per_country['country'])
                                                            .transform(lambda x:x.ffill()))

# Filtering na values
total_per_country.dropna(inplace=True)

df_southa = total_per_country[total_per_country['country'].isin(['Argentina','Bolivia','Brasil','Chile','Colombia','Ecuador','Paraguay','Peru','Uruguay','Venezuela'])]
df_northa = total_per_country[total_per_country['country'].isin(['Canada','Mexico','United States'])]

In North America, the United States had a faster progress until it reached 45 people fully vaccinated per hundred. Starting June, Canada starts significantly increasing its proportion of vaccinated people. Mexico has had a lower progress in the region.

In [ ]:
# North America
fig = px.line(df_northa, x='date', y='people_fully_vaccinated_per_hundred', color='country', text='country',
             width=800,
             height=600)

fig.update_traces(
    mode='markers+lines',
    hovertemplate='<b>%{text}</b><br>'+
                  'Fully vaccinated per 100: <b>%{y:.2f}</b><br>'+
                  '<extra>%{x}</extra>'
    )
fig.update_layout(
    title = 'Vaccination progress in North America',
    title_font_size=20,
    hoverlabel=dict(font_size=12), 
    hovermode='x'
    )
fig.update_yaxes(
    title="People fully vaccinated per hundred"
    )
fig.show()

In South America, there are two groups of countries. On one side, Chile and Uruguay are the first countries that went over 50 people fully vaccinated per hundred in this region. The rest of countries have had a similar trend since the start of vaccination, being under 25 per hundred at the beginning of August.

In [ ]:
# South America
fig = px.line(df_southa, x='date', y='people_fully_vaccinated_per_hundred', color='country', text='country',
             width=800,
             height=600)

fig.update_traces(
    mode='markers+lines',
    hovertemplate='<b>%{text}</b><br>'+
                  'Fully vaccinated per 100: <b>%{y:.2f}</b><br>'+
                  '<extra>%{x}</extra>'
    )
fig.update_layout(
    title = 'Vaccination progress in South America',
    title_font_size=20,
    hoverlabel=dict(font_size=12), 
    hovermode='x'
    )
fig.update_yaxes(
    title="People fully vaccinated per hundred"
    )
fig.show()

### **Vaccines around the world**

One of the main challenges during vaccination has been its access. There are several vaccines being produced and it can be interesting to see where each vaccine is being used. Some vaccines are used exclusively by a single country.

In [ ]:
### Vaccines per country
vaccine_per_country = df[['country', 'vaccines']].copy()
vaccine_per_country.drop_duplicates(keep='first', inplace=True)

#breaking 'vaccines' into different rows
vaccine_series = vaccine_per_country['vaccines'].str.split(', ').apply(pd.Series,1).stack()
vaccine_series.index = vaccine_series.index.droplevel(-1)
vaccine_series.name = 'vaccines'

vaccine_per_country.drop(columns=['vaccines'], inplace=True)
vaccine_per_country = vaccine_per_country.join(vaccine_series)

#getting iso code for countries
vaccine_per_country = pd.merge(vaccine_per_country, df[['country','iso_code']].copy().drop_duplicates(), 
                            how='left', on='country')
vaccine_per_country['valz'] = 1

### Choropleth map
vaccines_list = vaccine_per_country['vaccines'].drop_duplicates().to_list()
visible = np.array(vaccines_list)
vaccine_per_country['text'] = 'Country: ' + vaccine_per_country['country'] + '<br>' + 'Vaccine: ' + vaccine_per_country['vaccines']

traces = []
buttons = []
for vac in vaccines_list:
    traces.append(go.Choropleth(
        locations = vaccine_per_country.loc[vaccine_per_country.vaccines==vac]['iso_code'],
        z = vaccine_per_country.loc[vaccine_per_country.vaccines==vac]['valz'],
        coloraxis = 'coloraxis',
        colorscale = 'Blues',
        autocolorscale = False,
        marker_line_color = 'white',
        hovertemplate=vaccine_per_country.loc[vaccine_per_country.vaccines==vac]['text'] + '<extra></extra>',
        visible = True if vac == vaccines_list[0] else False))
    
    buttons.append(dict(label=vac,
                        method='update',
                        args=[{'visible':list(visible==vac)},
                              {'title':f'<b>Countries using {vac}</b>'}]))

updatemenus = [dict(type = 'dropdown',
                    active = 0,
                    showactive=True,
                    direction = 'down', 
                    xanchor = 'left', 
                    yanchor = 'top', 
                    x = 0, 
                    y = 1.09, 
                    font = dict(size=9, color='#000000'),
                    buttons = buttons)
               ]

fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus,
                            coloraxis=dict(colorscale='Blues')))

first_title = vaccines_list[0]
fig.update_layout(title=f"<b>Countries using {first_title}</b>",title_x=0.5,
                  coloraxis_showscale=False,
                  showlegend = False,
                  width = 800,
                  height = 600)

fig.show()